In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
mpl.style.use('ggplot')

In [ ]:
melt_file = 'IN_fullbasin_meltcontributionstytype_500m.txt'
#precip_file = 'IN_v01_fullbasinmasks.MERRA_rainfall_km3.annual_mean.by_elev.txt'
precip_file = 'IN_v01_fullbasinmasks.MERRA_rain_snow_fall_km3.annual_mean.by_elev.v2.csv'

melt_df = pd.read_csv(melt_file, sep='\s+', \
                      names=['elev', 'exposed_glacier_ice', 'snow_on_ice', 'snow_on_land'], \
                      index_col='elev', skiprows=1)
print(melt_df)

prec_df = pd.read_csv(precip_file, sep=',', names=['elev', 'rain', 'snow'], index_col='elev', skiprows=1)
print(prec_df)




In [ ]:
melt_cum = melt_df.sort_index(ascending=False).cumsum()
prec_cum = prec_df.sort_index(ascending=False).cumsum()

print(melt_cum)
print(prec_cum)

In [ ]:
all_contribs_model = pd.concat((melt_cum, prec_cum['rain']), axis=1)
all_contribs_merra = pd.concat((melt_cum['exposed_glacier_ice'], prec_cum), axis=1)

all_contribs_model = all_contribs_model[['exposed_glacier_ice', 'snow_on_ice', 'snow_on_land', 'rain']]
all_contribs_merra = all_contribs_merra[['exposed_glacier_ice', 'snow', 'rain']]

print(all_contribs_model)
print(all_contribs_merra)

In [ ]:
# Calculate contributions as fraction of total.  Drop all rows with all zeros first.

all_contribs_model = all_contribs_model[(all_contribs_model.T != 0).any()]        # drop zero rows
all_contribs_merra = all_contribs_merra[(all_contribs_merra.T != 0).any()]        # drop zero rows

sums_model = all_contribs_model.sum(axis=1)
sums_merra = all_contribs_merra.sum(axis=1)
print("SUMS, MODEL\n", sums_model)
print("SUMS, MERRA\n", sums_merra)
all_contribs_model_fraction = all_contribs_model.divide(sums_model.values, axis='index')
all_contribs_model_fraction.index.name = 'Elevation (m)'
all_contribs_merra_fraction = all_contribs_merra.divide(sums_merra.values, axis='index')
all_contribs_merra_fraction.index.name = 'Elevation (m)'

print("FRACTIONS, MODEL")
print(all_contribs_model_fraction)
print("FRACTIONS, MERRA")
print(all_contribs_merra_fraction)

In [ ]:
all_contribs_model_fraction.sum(axis=1)
all_contribs_merra_fraction.sum(axis=1)

In [ ]:
colors_model = ['r', 'blue', 'cyan', 'gray']
colors_merra = ['r', 'cyan', 'gray']

In [ ]:
fig1, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15,3))

ax = all_contribs_model_fraction.plot.bar(ax=ax1, title='Streamflow fraction', color=colors_model, stacked=True)
ax.set_ylabel('Fractional stream contribution')
ax = all_contribs_merra_fraction.plot.bar(ax=ax2, title='Streamflow fraction', color=colors_merra, stacked=True)
ax.set_ylabel('Fractional stream contribution')

In [ ]:
fig1, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15,3))
ax = all_contribs_model.plot(ax=ax1, title='Streamflow volume with distance down-river', color=colors_model)
ax.set_ylabel(r'Stream contribution (km$^3$)')
ax = all_contribs_merra.plot(ax=ax2, title='Streamflow volume with distance down-river', color=colors_merra)
ax.set_ylabel(r'Stream contribution (km$^3$)')

In [ ]:
# Plot modeled snow_on_land+snow_on_ice vs merra snow
model_snows = all_contribs_model['snow_on_land'] + all_contribs_model['snow_on_ice']
print("MODEL\n", model_snows)

In [ ]:
merra_snow = all_contribs_merra['snow']
merra_snow = merra_snow[merra_snow.index <= 6000]

print("MERRA\n", merra_snow)

In [ ]:
plt.plot(model_snows, merra_snow)

In [ ]:
snows_both = pd.concat((model_snows, merra_snow), axis=1)
snows_both.columns = ['sol+soi', 'merra_snow']
print(snows_both)

In [ ]:
snows_both.plot()